In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import inv,norm
from scipy.special import beta,betainc
from sklearn.linear_model import LassoLars, lars_path
import scipy

In [2]:
# import mean and covariance matrix of the 100 assets and 3 factors
mu = pd.read_csv('mean.csv',header=None)
S = pd.read_csv('var.csv',header=None)
mu = mu.drop(0,axis=1)
S = S.drop(0,axis=1)
mu = np.asarray(mu).flatten()
S = np.asarray(S)

# Set parameters for the optimization problem
sigma = 0.04   # target standard deviation
N = 103        # number of assets 
K = 3          # number of factors
T = 240        # number of time series observations
K_fold = 10     # number of folds for cross-validation
nobs = 100     # number of simulations

In [3]:
os_s = np.zeros(nobs)
os_sr = np.zeros(nobs)
for i in range(nobs):
    r = np.random.multivariate_normal(mu, S, T)
    muh = np.mean(r, axis = 0)
    sh = np.cov(r, rowvar = False, ddof = 0)
    wh = np.dot(inv(sh), muh)
    wh = sigma/np.sqrt(np.dot(wh,muh))*wh
    # print out wh
    os_s[i] = np.sqrt(np.dot(wh,S.dot(wh)))
    os_sr[i] = np.sqrt(12)*np.dot(wh,mu)/os_s[i]

In [4]:
# Average out of sample std dev across simulations and the sharpe ratio
# Followed by the Maximum Sharpe Ratio
print("Std dev:", np.mean(os_s))
print("Sharpe Ratio:", np.mean(os_sr) )
print("Theortical Maximum Sharpe Ratio:", np.sqrt(12*np.dot(mu,inv(S).dot(mu))))

Std dev: 0.07014086458178607
Sharpe Ratio: 0.9360672029432878
Theortical Maximum Sharpe Ratio: 1.8824247694760523


In [5]:
def theta_adj(theta,N,T):      # from smple code, needed for maxser
    a = N/2
    b = (T-N)/2
    theta_a = ((T-N-2)*theta-N)/T+2*theta**a*(1+theta)**(-(T-2)/2)/T/(betainc(a,b,theta/(1+theta))*beta(a,b))
    return theta_a

In [6]:
def MAXSER(r, sigma,Kfold):
    T,N = r.shape
    muh = np.mean(r, axis = 0)
    # from sample code
    sh = np.cov(r, rowvar = False, ddof = 0)
    theta = np.dot(muh,inv(Sh).dot(muh))
    theta_a = theta_adj(theta, N, T)
    rc = (1+theta_a)/np.sqrt(theta_a)*sigma
    zeta = np.zeros(Kfold)
    T1 = np.fix(T/Kfold).astype(int)
    y1 = rc*np.ones(T-T1)
    
    #find optimal Zeta with K-fold cross validation 
    for i in range(Kfold):
        ind = range(T1*i, (i+1)*T1)
        r1 = np.delete(r, ind, axis = 0)
        _, _, b1 = lars_path(r1, y1, method = 'lasso')
        q = np.std(np.matmul(r[ind, :], b1), axis = 0, ddof = 1)
        j = np.argmin(np.abs(sigma-q))
        zeta[i] = norm(b1[:,j], 1)/norm(b1[:, -1], 1)
    zeta = np.mean(zeta)
    y = rc*np.ones(T)
    _,_, b = lars_path(r,y, method= 'lasso')
    zeta0 = norm(b,1,axis = 0)/norm(b[:,-1],1)
    ind = np.min(np.argwhere(zeta < zeta0))
    q = (zeta - zeta0[ind-1])/(zeta0[ind] - zeta0[ind-1])
    w = (1-q)*b[:, ind-1]+q*b[:, ind]
    return w

In [7]:
os_s_mx= np.zeros(nobs)
os_sr_mx= np.zeros(nobs)
for i in range(nobs):
    # from demo code
    r = np.random.multivariate_normal(mu,S,T)   
    muh = np.mean(r,axis=0)
    Sh = np.cov(r,rowvar=False,ddof=0)
    
    series = MAXSER(r,sigma,K_fold)
    os_s_mx[i] = np.sqrt(np.dot(series,S.dot(series)))
    os_sr_mx[i] = np.sqrt(12)*np.dot(series,mu)/os_s_mx[i]

# Average out of sample std dev across simulations and the sharpe ratio
# Followed by the Maximum Sharpe Ratio
print("Std dev:", np.mean(os_s_mx))
print("Sharpe Ratio:", np.mean(np.mean(os_sr_mx)) )

Std dev: 0.04078653518625468
Sharpe Ratio: 1.249525638777942


In [8]:
def ridge(alpha,trset,valset,sigma = 0.04):   
    
    T,N = trset.shape
    muh = np.mean(trset,axis=0)
    Sh = np.cov(trset,rowvar=False,ddof=0)
    theta = np.dot(muh,inv(Sh).dot(muh))
    theta_a = theta_adj(theta,N,len(trset))    
    rc = (1+theta_a)/np.sqrt(theta_a)*sigma
    y = rc*np.ones(T)
    w = np.matmul(inv(np.matmul(trset.T,trset)+ np.eye(N)*alpha),np.matmul(trset.T,y))
    rh = valset.dot(w)
    os_s = np.std(rh, ddof =1)
    return os_s  

In [9]:
def findridge(alpha,trset,valset,sigma):
    return ridge(alpha,trset,valset) - sigma

#need to find opitmal alpha now

def Ridge_function(r,sigma,K_fold):
    """
    Performs K_fold validation to find the best alpha - tuning parameter- for the ridge regression for every fold,
    then averages those best alphas to return the optimal alpha.
    """
    
    alphas=[]
    
    T,N = r.shape
    muh = np.mean(r,axis=0)
    Sh = np.cov(r,rowvar=False,ddof=0)
    theta = np.dot(muh,inv(Sh).dot(muh))
    theta_a = theta_adj(theta,N,T)   
    rc = (1+theta_a)/np.sqrt(theta_a)*sigma
    zeta = np.zeros(K_fold)
    T1 = np.fix(T/K_fold).astype(int)    #number of data in each fold
    y1 = rc*np.ones(T-T1)
    
    #use K-fold cross validation to find the optimal zeta
    for i in range(K_fold):
        ind = range(i*T1,(i+1)*T1)     #index of beginninG and end of the selected fold 
        r1 = np.delete(r,ind,axis=0)  # rest of data after deleting the selected fold
        
        best_alpha = scipy.optimize.brentq(f, 0,50, args=(r1,r[ind,:],sigma))    
        alphas.append(best_alpha)
    
    opt_alpha = sum(alphas) / len(alphas)
    return opt_alpha